In [18]:
import json
from pathlib import Path

import pandas as pd
from fastai.vision.all import cnn_learner
from fastai.vision.data import ImageDataLoaders
from fastai.vision.models import xresnet

ROOT_PATH = Path("..")

In [19]:
# Setup y_train, y_test
df = pd.read_csv(ROOT_PATH / "data/raw/metadata.csv")
svc_ids = pd.read_json(ROOT_PATH / "data/raw/song_vs_call.json").squeeze()
svc_df = df.loc[df.id.isin(svc_ids)].copy()

with open(ROOT_PATH / "data/processed/svc_split.json") as svc_split_file:
    svc_split = json.load(svc_split_file)
    train_ids = svc_split["train_ids"]
    test_ids = svc_split["test_ids"]

# Add response variable
type_col = svc_df.type.str.lower().str.replace(" ", "").str.split(",")
filtered_type_col = type_col.apply(lambda l: set(l) - {"call", "song"})
svc_df["label"] = type_col.apply(lambda l: "call" in l).astype(int)

y_df = svc_df.reindex(columns=["id", "pred"]).copy()
y_train, y_test = (
    y_df[y_df.id.isin(train_ids)].drop(columns=["id"]).squeeze(),
    y_df[y_df.id.isin(test_ids)].drop(columns=["id"]).squeeze(),
)

In [20]:
svc_df["name"] = svc_df.id.astype(str) + ".png"
svc_df["is_valid"] = svc_df.id.isin(test_ids)
image_df = (
    svc_df.reindex(columns=["id", "name", "label", "is_valid"]).set_index("id").copy()
)

In [23]:
bs = 64  # Batch size
data = (
    # convert_mode is passed on intern|ally to the relevant function that will handle converting the images;
    # 'L' results in one color channel
    ImageDataLoaders.from_df(
        image_df, folder=ROOT_PATH / "data/raw/sonograms", valid_col="is_valid"
    )
    # .databunch(bs=bs).normalize()
)
learn = cnn_learner(data, xresnet.xresnet34)

In [ ]:
learn.fine_tune(1)